# Four-Circle example (E4C)

This is a sample measured at APS 33BM using SPEC.
Let's compare the *SPEC* values with *hklpy*.

```
sample: LNO_LAO
crystal:  3.781726143 3.791444574 3.79890313 90.2546203 90.01815424 89.89967858
geometry: fourc
mode: 0 (Omega equals zero)
lambda: 1.239424258
r1: (0, 0, 2) 38.09875 19.1335 90.0135 0
r2: (1, 1, 3) 65.644 32.82125 115.23625 48.1315
Q: (2, 2, 1.9) 67.78225 33.891 145.985 48.22875 -0.001 -0.16
UB: -1.658712442 0.09820024135 -0.000389705578
    -0.09554990312 -1.654278629 0.00242844486
    0.0002629818914 0.009815746824 1.653961812

#F /.../LNO_LAO
#E 1276730676
#D Wed Jun 16 18:24:36 2010
#C LNO_LAO  User = epix33bm
#O0  2-theta     theta       chi       phi   antheta  an2theta    z-axis     m_1_8
#O1      xt1       yt1       yt2       yt3     m_2_5       zt1      samx      samz
#O2 DCM.theta       wst       wsl       wsb       wsr       m22  
#G0 0 0 1 0 0 1 0 0 0 0 0 0 50 0 0.1 0 68 68 50 -1 1 1 3.13542 3.13542 0 463.6 838.8
#G1 3.781726143 3.791444574 3.79890313 90.2546203 90.01815424 89.89967858 1.661462253 1.657219786 1.65396364 89.74541108 89.98229138 90.10024173 0 0 2 1 1 3 38.09875 19.1335 90.0135 0 0 0 65.644 32.82125 115.23625 48.1315 0 0 1.239424258 1.239424258
#G3 -1.658712442 0.09820024135 -0.000389705578 -0.09554990312 -1.654278629 0.00242844486 0.0002629818914 0.009815746824 1.653961812
#G4 1.999995696 1.999999878 1.899998938 1.239424258 18.09198724 18.2076099 0 89.92021031 0.7 0 0 0 0 0 0 0 -180 -180 -180 -180 -180 -180 -180 -180 -180 0
#Q 2 2 1.9
#P0 67.78225 33.891 145.985 48.22875 -0.001 -0.16 2.5 -0.2
#P1 -3.4480499 0.49927508 0.030010083 0.499275 1.749425 -113.52071 -946.48814 0
#P2 11.508019 2.9999781 30.000019 2.9999781 29.999854 1230.0415 
#U DCM_energy 9.99904
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import logging
logger = logging.getLogger("notebook")
logger.setLevel(logging.INFO)

import gi
gi.require_version('Hkl', '5.0')

from hkl.diffract import E4CV
from hkl.util import Lattice

from bluesky import plans as bp, RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback

from ophyd import Component
from ophyd import PseudoSingle
from ophyd import SoftPositioner

from apstools.diffractometer import Constraint, DiffractometerMixin

import pyRestTable

RE = RunEngine({})
bec = BestEffortCallback()
RE.subscribe(bec)

0

In [2]:
# describe the diffractometer
class FourCircleDiffractometer(DiffractometerMixin, E4CV):
    h = Component(PseudoSingle, '', labels=("hkl", "fourc"))
    k = Component(PseudoSingle, '', labels=("hkl", "fourc"))
    l = Component(PseudoSingle, '', labels=("hkl", "fourc"))

    omega = Component(SoftPositioner, labels=("motor", "fourc"))
    chi   = Component(SoftPositioner, labels=("motor", "fourc"))
    phi   = Component(SoftPositioner, labels=("motor", "fourc"))
    tth   = Component(SoftPositioner, labels=("motor", "fourc"))

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # since this diffractometer uses simulated motors,
        # prime the SoftPositioners (motors) with initial values
        # otherwise, position == None --> describe, etc gets borked
        for axis in (self.omega, self.phi, self.chi, self.tth):
            axis.move(0)

## setup the diffractometer

In [3]:
# initialize the calculation engine
fourc = FourCircleDiffractometer('', name='fourc', labels=("diffractometer", "fourc"))
fourc.calc.engine.mode = "constant_omega"
fourc.energy.put(9.99904*0.1)     # keV
print(f"energy (keV): {fourc.energy.value}")
#print(f"energy (keV): {fourc.calc.energy}")
print(f"wavelength (A): {fourc.calc.wavelength}")

print(f"{fourc.name} modes: {fourc.engine.modes}")
print(f"selected mode: {fourc.calc.engine.mode}")

energy (keV): 0.9999040000000001
wavelength (A): 1.2399610362594808
fourc modes: ['bissector', 'constant_omega', 'constant_chi', 'constant_phi', 'double_diffraction', 'psi_constant']
selected mode: constant_omega


In [4]:
fourc.energy

Signal(name='fourc_energy', parent='fourc', value=0.9999040000000001, timestamp=1582851112.8946128)

## define the crystal

* sample: LNO_LAO
* crystal:  3.781726143 3.791444574 3.79890313 90.2546203 90.01815424 89.89967858

In [5]:
# define the crystal
fourc.calc.new_sample('LNO LAO', 
    lattice=Lattice(
        a=3.781726143, b=3.791444574, c=3.79890313, 
        alpha=90.2546203, beta=90.01815424, gamma=89.89967858))

HklSample(name='LNO LAO', lattice=LatticeTuple(a=3.781726143, b=3.791444574, c=3.79890313, alpha=90.2546203, beta=90.01815424, gamma=89.89967858), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), U=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), UB=array([[ 1.66146225e+00, -2.89938471e-03,  5.11196668e-04],
       [ 0.00000000e+00,  1.65721725e+00,  7.34922202e-03],
       [ 0.00000000e+00,  0.00000000e+00,  1.65394723e+00]]), reflections=[])

## Orient the crystal on the diffractometer

from the SPEC data file (tth, th, chi, phi):

* r1: (0, 0, 2) 38.09875 19.1335 90.0135 0
* r2: (1, 1, 3) 65.644 32.82125 115.23625 48.1315

In [6]:
# use two known reflections to define the orientation
p1 = fourc.calc.Position(
        tth=38.09875, omega=19.1335, chi=90.0135, phi=0)
p2 = fourc.calc.Position(
        tth=65.644, omega=32.82125, chi=115.23625, phi=48.1315)

# index each the reflections by (hkl)
r1 = fourc.calc.sample.add_reflection(
    0, 0, 2,
    position=p1)
r2 = fourc.calc.sample.add_reflection(
    1, 1, 3,
    position=p2)

# compute the orientation
fourc.calc.sample.compute_UB(r1, r2)

1

In [7]:
print("computed orientation UB matrix\n", fourc.calc.sample.UB)

print("compare with UB matrix computed in SPEC")
print("""
UB: -1.658712442 0.09820024135 -0.000389705578
    -0.09554990312 -1.654278629 0.00242844486
    0.0002629818914 0.009815746824 1.653961812
""")
print("same values, different row order")

computed orientation UB matrix
[[-9.55499011e-02 -1.65427863e+00  2.42844485e-03]
 [ 2.62981975e-04  9.81483906e-03  1.65396181e+00]
 [-1.65871244e+00  9.82002396e-02 -3.89705577e-04]]
compare with UB matrix computed in SPEC

UB: -1.658712442 0.09820024135 -0.000389705578
    -0.09554990312 -1.654278629 0.00242844486
    0.0002629818914 0.009815746824 1.653961812

same values, different row order


## Compare calculated values

In [8]:
# Q: (2, 2, 1.9) 67.78225 33.891 145.985 48.22875 -0.001 -0.16
# tth th chi phi
# Hey!  this (omega) is not zero!  Can't compute that with constant_omega mode

print("possible motor positions for each (hkl) reflection")
print(fourc.forwardSolutionsTable(
    (
        (2, 2, 1.9),
    ),
    full=True
))

possible motor positions for each (hkl) reflection
=========== ======== ======= ========= ======= =========
(hkl)       solution omega   chi       phi     tth      
=========== ======== ======= ========= ======= =========
(2, 2, 1.9) 0        0.00000 -42.37958 5.92845 -67.81565
(2, 2, 1.9) 1        0.00000 137.62042 5.92845 67.81565 
=========== ======== ======= ========= ======= =========



In [9]:
# check all available modes
for mode in fourc.engine.modes:
    fourc.calc.engine.mode = mode
    print(f"mode: {mode}")
    print(fourc.forwardSolutionsTable(
        (
            (2, 2, 1.9),
        ),
        full=True
    ))

mode: bissector
=========== ======== ========== ========== ========== =========
(hkl)       solution omega      chi        phi        tth      
=========== ======== ========== ========== ========== =========
(2, 2, 1.9) 0        -33.90782  -34.01497  48.22884   -67.81565
(2, 2, 1.9) 1        33.90782   34.01497   -131.77116 67.81565 
(2, 2, 1.9) 2        33.90782   145.98503  48.22884   67.81565 
(2, 2, 1.9) 3        -146.09218 -34.01497  48.22884   67.81565 
(2, 2, 1.9) 4        -33.90782  -145.98503 -131.77116 -67.81565
(2, 2, 1.9) 5        -146.09218 -145.98503 -131.77116 67.81565 
=========== ======== ========== ========== ========== =========

mode: constant_omega
=========== ======== ======= ========= ======= =========
(hkl)       solution omega   chi       phi     tth      
=========== ======== ======= ========= ======= =========
(2, 2, 1.9) 0        0.00000 -42.37958 5.92845 -67.81565
(2, 2, 1.9) 1        0.00000 137.62042 5.92845 67.81565 
=========== ======== ======= ========

In [10]:
fourc.calc.engine.mode = "bissector"

solutions = fourc.calc.forward((2, 2, 1.9))

In [11]:
# Q: (2, 2, 1.9) 67.78225 33.891 145.985 48.22875 -0.001 -0.16
spec = dict(tth=67.78225, omega=33.891, chi=145.985, phi=48.22875)

import math

for sol in solutions:
    sum = (sol.tth - spec["tth"])**2
    sum += (sol.omega - spec["omega"])**2
    sum += (sol.chi - spec["chi"])**2
    sum += (sol.phi - spec["phi"])**2
    rms = math.sqrt(sum)
    print(f"{sol.tth:10f}  {sol.omega:10f}  {sol.chi:10f}  {sol.phi:10f} : RMS={rms}")


-67.815649  -33.907825  -34.014970   48.228841 : RMS=235.33690755458932
 67.815649   33.907825   34.014970  -131.771159 : RMS=211.98409457888116
 67.815649   33.907825  145.985030   48.228841 : RMS=0.03739763689293413
 67.815649  -146.092175  -34.014970   48.228841 : RMS=254.54652563532719
-67.815649  -33.907825  -145.985030  -131.771159 : RMS=375.0065821698689
 67.815649  -146.092175  -145.985030  -131.771159 : RMS=387.35050061460566


## apply motion constraints

Restrict to only non-negative angles

In [12]:
diffractometer_constraints = {
    # axis:  Constraint(lo_limit, hi_limit, value, fit)
    "tth":   Constraint(0, 180, 0, True),
    "omega": Constraint(0, 180, 0, True),
    "chi":   Constraint(0, 180, 0, True),
    "phi":   Constraint(0, 180, 0, True),
}

fourc.applyConstraints(diffractometer_constraints)
print("revised constraints")
fourc.showConstraints()

revised constraints
===== ========= ========== ===== ====
axis  low_limit high_limit value fit 
===== ========= ========== ===== ====
omega 0.0       180.0      0.0   True
chi   0.0       180.0      0.0   True
phi   0.0       180.0      0.0   True
tth   0.0       180.0      0.0   True
===== ========= ========== ===== ====



In [13]:
print(fourc.forwardSolutionsTable(
    (
        (2, 2, 1.9),
    ),
    full=True
))
print(f"SPEC angles for (2, 2, 1.9): {spec}")

=========== ======== ======== ========= ======== ========
(hkl)       solution omega    chi       phi      tth     
=========== ======== ======== ========= ======== ========
(2, 2, 1.9) 0        33.90782 145.98503 48.22884 67.81565
=========== ======== ======== ========= ======== ========

SPEC angles for (2, 2, 1.9): {'tth': 67.78225, 'omega': 33.891, 'chi': 145.985, 'phi': 48.22875}


## Summary

The *Q* reflection computed by this *hklpy* code agrees with
the SPEC data in the file when the diffractometer mode is set to `bissector`.

That's so encouraging, let's spot-check a few more *Q* values from the data file.

The diffractometer mode can change for each scan.  SPEC does not report the values of each motor at every step of an (hkl) scan, only at the start.  The (hkl) are in the `#Q` line, the motor positions are the first four values of the `#P0` line.  Here are some *Q* values from various scans in the data file:

```
#Q 0.000274545 -0.000143088 2
#P0 38.084 19.133 90.0135 0 -0.001 -0.16 2.5 -0.2

#Q 0 0 4
#P0 81.46425 40.81625 90.0135 0 -0.001 -0.16 2.5 -0.2

#Q 0.962396 1.0366 2.99979
#P0 65.637 32.8185 115.20525 48.2855 -0.001 -0.16 2.5 -0.2

#Q 1.00133 1.00133 2.99945
#P0 65.644 32.82125 115.23625 48.1315 -0.001 -0.16 2.5 -0.2

#Q 2 2 2.00001
#P0 69.0675 34.53375 144.61725 48.2265 -0.001 -0.16 2.5 -0.2
```

In [14]:
reflections = (
        (0, 0, 2),
        (0.000274545, -0.000143088, 2),
        (0, 0, 4),
    )
fourc.phi.move(0)
fourc.calc.engine.mode ="constant_phi"
print(f"mode: {fourc.calc.engine.mode}")
print(fourc.forwardSolutionsTable(
    reflections,
    full=True
))

reflections = (
        (1, 1, 3),
        (0.962396, 1.0366, 2.99979),
        (1.00133, 1.00133, 2.99945),
        (2, 2, 2),
        (2, 2, 2.00001),
    )
fourc.calc.engine.mode ="bissector"
print(f"mode: {fourc.calc.engine.mode}")
print(fourc.forwardSolutionsTable(
    reflections,
    full=True
))


mode: constant_phi
============================== ======== ======== ======== ======= ========
(hkl)                          solution omega    chi      phi     tth     
============================== ======== ======== ======== ======= ========
(0, 0, 2)                      0        19.13472 90.01350 0.00000 38.10119
(0.000274545, -0.000143088, 2) 0        19.13836 90.02163 0.00000 38.10118
(0, 0, 4)                      0        40.83762 90.01350 0.00000 81.50699
============================== ======== ======== ======== ======= ========

mode: bissector
=========================== ======== ======== ========= ======== ========
(hkl)                       solution omega    chi       phi      tth     
=========================== ======== ======== ========= ======== ========
(1, 1, 3)                   0        32.83450 115.20291 48.13305 65.66899
(0.962396, 1.0366, 2.99979) 0        32.83454 115.20299 50.26719 65.66907
(1.00133, 1.00133, 2.99945) 0        32.83798 115.23632 48.13326 65.6

# Conclusion

The simulation shows comparable position calculations between *hklpy* and *SPEC* for the orientation of this crystal.  Different diffractometer modes were used at various scans in the file.